In [80]:
import pandas as pd
import matplotlib as plt

In [81]:
data = pd.read_csv('data.csv')


In [82]:
for i in range(len(data)):
    if (data.loc[i, 'Pressure (psi)'] < 27 or data.loc[i, 'Pressure (psi)'] > 150 or
        data.loc[i, 'Temperature (°C)'] > 88 or
        data.loc[i, 'Oil Level (%)'] < 18 or
        data.loc[i, 'Tool RPM (rpm)'] > 2500):
        data.loc[i, 'Machine Status'] = 'NotRunning'

In [83]:
data.head()

,Timestamp,Pressure (psi),Temperature (°C),Oil Flow (L/min),Vibration (mm/s),Power Status,Oil Level (%),Tool RPM (rpm),Machine Status,Current (A)
0,17-09-2023 00:00,45.2,29.8,2.4,0.021,ON,73.5,1505,Running,10.3
1,17-09-2023 00:00,48.2,28.8,3.4,0.031,ON,70.5,1805,Running,11.3
2,17-09-2023 00:01,45.1,29.9,2.5,0.022,ON,73.3,1508,Running,10.4
3,17-09-2023 00:01,48.1,28.9,3.5,0.032,ON,70.3,1808,Running,11.4
4,17-09-2023 00:02,45.3,30.0,2.6,0.023,ON,73.2,1512,Running,10.5


In [84]:
data.to_csv('data_aug.csv', index=False) 

In [85]:
data['Vibration (mm/s)'].median()

0.6

In [86]:
plt.figure(figsize=(20,50))
ax=data['time_cycles'].plot(kind='barh',width=0.8, stacked=True,align='center')

AttributeError: module 'matplotlib' has no attribute 'figure'